## Below are our imports for our data cleaning, exploratory data analysis and modeling.

In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold, cross_validate, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.linear_model import Lasso, Ridge, RidgeCV, LassoCV
from sklearn.linear_model import LinearRegression

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
freedom_happiness_df = pd.read_csv('data/P5clean.csv')

## Test Linear Regression modeling

In [3]:
# #drop all nulls and duplicates
# #        We'll liekly want to revisit and do some more precise cleaning, 
# #        but in the interest of viewing correleations and establishing 
# #        baseline performance we'll just drop everything for now

# # Any duplicate rows? No
len(freedom_happiness_df[freedom_happiness_df.duplicated()])
# drop nulls
freedom_happiness_df.dropna(inplace=True)
freedom_happiness_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1682 entries, 0 to 1681
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   year                              1682 non-null   float64
 1   overall_score                     1682 non-null   float64
 2   property_rights                   1682 non-null   float64
 3   government_integrity              1682 non-null   float64
 4   tax_burden                        1682 non-null   float64
 5   government_spending               1682 non-null   float64
 6   business_freedom                  1682 non-null   float64
 7   labor_freedom                     1682 non-null   float64
 8   monetary_freedom                  1682 non-null   float64
 9   trade_freedom                     1682 non-null   float64
 10  investment_freedom                1682 non-null   float64
 11  financial_freedom                 1682 non-null   float64
 12  log_gd

In [4]:
X = freedom_happiness_df.drop(columns=['year', 'life_ladder', 'overall_score', 'country_name'])
y = freedom_happiness_df['life_ladder']

In [5]:
# Linear regression (using ALL features to predict 'overall_score', not just economic features)

# assign
# exclude = ['country_name', 'life_ladder']
# X_features = [i for i in freedom_happiness_df if i not in exclude]
# X = freedom_happiness_df[X_features]
# y = freedom_happiness_df['life_ladder']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

# scale
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

# fit / score
lm = linear_model.LinearRegression(n_jobs = -1)

lm.fit(X_train, y_train)
print(f'Trainging R-squared: {lm.score(X_train, y_train)}')
print(f'Testing R-squared: {lm.score(X_test, y_test)}')

lm.fit(X_train_sc, y_train)
print(f'Scaled trainging R-squared: {lm.score(X_train_sc, y_train)}')
print(f'Scaled testing R-squared: {lm.score(X_test_sc, y_test)}')

Trainging R-squared: 0.7677704554917241
Testing R-squared: 0.7768002853217872
Scaled trainging R-squared: 0.7677704554917241
Scaled testing R-squared: 0.7768002853217872


In [6]:
freedom_happiness_df.corr()['life_ladder'].sort_values(ascending=False)

life_ladder                         1.000000
log_gdp_per_capita                  0.786262
healthy_life_expectancy_at_birth    0.740277
social_support                      0.709317
government_integrity                0.684178
property_rights                     0.658343
business_freedom                    0.601996
overall_score                       0.589604
financial_freedom                   0.550077
trade_freedom                       0.526725
positive_affect                     0.520366
choice_freedom                      0.515615
investment_freedom                  0.459059
monetary_freedom                    0.294137
labor_freedom                       0.213469
generosity                          0.180719
year                                0.046073
tax_burden                         -0.279132
negative_affect                    -0.301800
government_spending                -0.428348
perceptions_of_corruption          -0.430875
Name: life_ladder, dtype: float64

In [7]:
freedom_happiness_df.columns

Index(['year', 'overall_score', 'property_rights', 'government_integrity',
       'tax_burden', 'government_spending', 'business_freedom',
       'labor_freedom', 'monetary_freedom', 'trade_freedom',
       'investment_freedom', 'financial_freedom', 'log_gdp_per_capita',
       'social_support', 'healthy_life_expectancy_at_birth', 'choice_freedom',
       'generosity', 'perceptions_of_corruption', 'positive_affect',
       'negative_affect', 'country_name', 'life_ladder'],
      dtype='object')

In [8]:
# Linear regression (using exclusively economic features to predict happiness)

#assign
exclude = ['life_ladder', 'country_name', 'year', 'overall_score', 'country_year', 'judicial_effectiveness', 'fiscal_health']
econ_cols = ['property_rights', 'government_integrity',
       'tax_burden', 'government_spending', 'business_freedom',
       'labor_freedom', 'monetary_freedom', 'trade_freedom',
       'investment_freedom', 'financial_freedom']
happiness_cols = ['log_gdp_per_capita',
       'social_support', 'healthy_life_expectancy_at_birth', 'choice_freedom',
       'generosity', 'perceptions_of_corruption', 'positive_affect',
       'negative_affect', 'country_name', 'life_ladder']

X_features = [i for i in econ_cols if i not in exclude]
# X_features = econ_freedom_df.drop(columns = ['country_name', 'life_ladder', 'country_year'])
X2 = freedom_happiness_df[X_features]
y2 = freedom_happiness_df['life_ladder']
X_train, X_test, y_train, y_test = train_test_split(X2, y2, random_state = 42)

# scale
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

# fit / score
lm = linear_model.LinearRegression(n_jobs = -1)

lm.fit(X_train, y_train)
print(f'Trainging R-squared: {lm.score(X_train, y_train)}')
print(f'Testing R-squared: {lm.score(X_test, y_test)}')

lm.fit(X_train_sc, y_train)
print(f'Scaled trainging R-squared: {lm.score(X_train_sc, y_train)}')
print(f'Scaled testing R-squared: {lm.score(X_test_sc, y_test)}')

Trainging R-squared: 0.5176527266483792
Testing R-squared: 0.571910564905197
Scaled trainging R-squared: 0.5176527266483792
Scaled testing R-squared: 0.571910564905197


In [9]:
# Linear regression (using exclusively happiness features to predict happiness)

# assign
exclude = ['life_ladder', 'country_name', 'year', 'overall_score', 'country_year', 'judicial_effectiveness', 'fiscal_health']
X_features = [i for i in happiness_cols if i not in exclude]
X3 = freedom_happiness_df[X_features]
y3 = freedom_happiness_df['life_ladder']
X_train, X_test, y_train, y_test = train_test_split(X3, y3, random_state = 42)

# scale
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

# fit / score
lm = linear_model.LinearRegression(n_jobs = -1)

lm.fit(X_train, y_train)
print(f'Trainging R-squared: {lm.score(X_train, y_train)}')
print(f'Testing R-squared: {lm.score(X_test, y_test)}')

lm.fit(X_train_sc, y_train)
print(f'Scaled trainging R-squared: {lm.score(X_train_sc, y_train)}')
print(f'Scaled testing R-squared: {lm.score(X_test_sc, y_test)}')

Trainging R-squared: 0.7563467159669359
Testing R-squared: 0.7582342047514951
Scaled trainging R-squared: 0.7563467159669359
Scaled testing R-squared: 0.7582342047514955


In [10]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=.2, random_state=41)
#Instantiate the estimator
lr = LinearRegression(n_jobs=-1)

#fit estimator
lr.fit(X_train, y_train)

#predict to score
odd=lr.predict(X_test)

# The intercept and Coefficients
print('Coefficient values of the Model are ', lr.coef_)
print('Intercept value is ', lr.intercept_)

Coefficient values of the Model are  [-9.51392991e-04 -2.46037977e-03 -6.10654731e-03 -1.79073962e-03
  1.70574055e-03 -3.13302768e-03  1.68962193e-03 -7.17713405e-03
  1.50911622e-03  3.35278249e-03  3.74228605e-01  2.12374903e+00
  2.64441263e-02  3.19093075e-01  5.24142204e-01 -6.20544086e-01
  1.85619855e+00  1.27616792e-01]
Intercept value is  -1.5974285775967276


In [11]:
ntr=cross_val_score(lr, X_train, y_train, cv=5, n_jobs = -1) 
print(ntr)
f'{round(np.mean(ntr), 2)} \u00B1 {round(2 * np.std(ntr), 2)}'
#I believe there are outliers and that my model is overfit

[0.77897998 0.80691654 0.76764847 0.74254582 0.73999068]


'0.77 ± 0.05'

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
#scale
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

# fit LassoCV
lsv=LassoCV(cv = 10)
lsv.fit(X_train_sc, y_train)
oddlasso=lsv.predict(X_test_sc)

#R2 score on Xtrain
print('Xtrainscore:', lsv.score(X_train_sc, y_train))


Xtrainscore: 0.7656583339455707


In [13]:
l_alphas = np.arange(0.01, .10, 0.05)

def lasso_coefs(X, Y, alphas):
    coefs = []
    lasso_reg = Lasso()
    for a in alphas:
        lasso_reg.set_params(alpha=a)
        lasso_reg.fit(X, Y)
        coefs.append(lasso_reg.coef_)
    return coefs

# model using lasso_coefs function from above
l_coefs = lasso_coefs(X_train_sc, y_train, l_alphas)

features_coefs_df = pd.DataFrame(l_coefs, columns =X.columns)
features_coefs_df['alpha'] = l_alphas
for col, coef in features_coefs_df.iloc[features_coefs_df.index.max()]\
                              .iteritems():
    if not coef:
        print(col, coef)

property_rights 0.0
government_spending -0.0
business_freedom 0.0
labor_freedom -0.0
monetary_freedom 0.0
trade_freedom 0.0
investment_freedom 0.0
negative_affect -0.0


In [23]:
columns_to_filter_out = [col for col, coef in features_coefs_df\
                                              .iloc[features_coefs_df.index.max()]\
                                              .iteritems() if not coef]
listoffeatures=['property_rights', 'government_integrity',
       'tax_burden', 'government_spending', 'business_freedom',
       'labor_freedom', 'monetary_freedom', 'trade_freedom',
       'investment_freedom', 'financial_freedom', 'log_gdp_per_capita',
       'social_support', 'healthy_life_expectancy_at_birth', 'choice_freedom',
       'generosity', 'perceptions_of_corruption', 'positive_affect',
       'negative_affect']
freedom_features = [col for col in listoffeatures\
                      if col not in columns_to_filter_out]

In [24]:
len(freedom_features)

10

In [25]:
freedom_features

['government_integrity',
 'tax_burden',
 'financial_freedom',
 'log_gdp_per_capita',
 'social_support',
 'healthy_life_expectancy_at_birth',
 'choice_freedom',
 'generosity',
 'perceptions_of_corruption',
 'positive_affect']

In [26]:
#Instantiate
ridge_reg = RidgeCV()

#Fit the model
ridge_reg.fit(X_train_sc, y_train)

#Predict
oddridge=ridge_reg.predict(X_test_sc)

# X_train_sc R2 score below
print(ridge_reg.score(X_train_sc, y_train))

0.7677115697287443


In [27]:
# X=StandardScaler().fit_transform(X)

rf=RandomForestRegressor()
rf_scores=cross_val_score(rf, X, y, cv=KFold(n_splits=10, shuffle=True, random_state=42))
print('RF', round (rf_scores.mean(), 3), '+-', round (2 *rf_scores.std(), 3))


# rfpredictions= cross_val_predict(rf, X, y, cv=KFold(n_splits=5, shuffle=True, random_state=42))


RF 0.882 +- 0.019


In [28]:
# # rfpipe=Pipeline([('ss', StandardScaler()),
# #                 ('rf', RandomForestRegressor())])
# pipe_param={'rf__n_estimators':[10,25, 50, 100],
#     'rf__criterion':['mse'],
#     'rf__max_depth':[None],
#     'rf__min_samples_split':[2,5,10,15],
#     'rf__min_samples_leaf':[1,3,5,7],
#     'rf__min_weight_fraction_leaf':[0,1,2],
#     'rf__max_features':['auto','sqrt','log2',None],
#     'rf__max_leaf_nodes':[None],
#     'rf__min_impurity_decrease':[0],
#     'rf__min_impurity_split':[None],
#     'rf__bootstrap':[True],
#     'rf__oob_score': [True,False],
#     'rf__n_jobs':[-1],
#     'rf__random_state': [42],
#     'rf__verbose':[0],
#     'rf__warm_start':[False,True],
#     'rf__ccp_alpha':[0],
#     'rf__max_samples':[None]}

# # rfpipe_gridsearcher=GridSearchCV(rf, pipe_param, verbose=2)

# rf_gridsearcher=GridSearchCV(rf, pipe_param, verbose=0)


# rf_gridsearcher.fit(X,y)

# rf_gridsearcher.param_grid

# rf_gridsearcher.best_score_

# rf_gridsearcher.best_estimator_

# rf_gridsearcher.best_params_

# rf_gridsearcher.grid_scores_

In [29]:
%%timeit
# rfpipe=Pipeline([('ss', StandardScaler()),
#                 ('rf', RandomForestRegressor())])
# pipe_param={'max_depth':[2, 5, 10, None]}

pipe_param={'n_estimators':[50, 100],
            'criterion':["mse"],
            'max_depth':[None],
            'min_samples_split':[2,5],
            'min_samples_leaf':[1,3,5],
            'min_weight_fraction_leaf':[0,1,2],
            'max_features':['auto','sqrt','log2',None]}


# n_estimators=100,
#     *,
#     criterion='mse',
#     max_depth=None,
#     min_samples_split=2,
#     min_samples_leaf=1,
#     min_weight_fraction_leaf=0.0,
#     max_features='auto',
    
    
# rfpipe_gridsearcher=GridSearchCV(rf, pipe_param, verbose=2)

rf_gridsearcher=GridSearchCV(rf, pipe_param, verbose=1)

rf_gridsearcher.fit(X,y)

rf_gridsearcher.param_grid

print(rf_gridsearcher.best_score_)

print(rf_gridsearcher.best_estimator_)

print(rf_gridsearcher.best_params_)


Fitting 5 folds for each of 144 candidates, totalling 720 fits


KeyboardInterrupt: 

In [ ]:
RandomForestRegressor().get_params().keys()

In [30]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42)

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [31]:
def model_fn(layer_one_neurons=32, layer_two_neurons=32, d_drop1=.25):
    model = Sequential()
    model.add(Dense(layer_one_neurons, activation='relu', input_shape=(18,),kernel_regularizer=l2(0.001)))
    model.add(Dropout(d_drop1))
    model.add(Dense(layer_two_neurons,activation='relu'))
    model.add(Dense(1, activation=None))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

nn = KerasRegressor(build_fn=model_fn, epochs=1000, batch_size=512, verbose=0)
#scikilt learn complied model--CV is reduced t o 2 for time-do not do in work
params={ 'epochs':[20],
        'layer_one_neurons':[32],
       'layer_two_neurons':[32],
      'd_drop1': [.25]}
gs= GridSearchCV(nn, param_grid=params, cv=5)
gs.fit(X_train_sc, y_train)
print(gs.best_score_)
gs.best_params_

-11.552700996398926


{'d_drop1': 0.25,
 'epochs': 20,
 'layer_one_neurons': 32,
 'layer_two_neurons': 32}

In [32]:
pred= gs.predict(X_test_sc)
metrics.r2_score(pred, y_test)

-0.5076325419382908

In [33]:

G=X[freedom_features]
X_train, X_test, y_train, y_test= train_test_split(G, y, test_size=.2, random_state=41)
#Instantiate the estimator
lr = LinearRegression(n_jobs=-1)

#fit estimator
lr.fit(X_train, y_train)

#predict to score
odd=lr.predict(X_test)

# The intercept and Coefficients
print('Coefficient values of the Model are ', lr.coef_)
print('Intercept value is ', lr.intercept_)

ntr=cross_val_score(lr, G, y, cv=5, n_jobs = -1) 
print(ntr)
f'{round(np.mean(ntr), 2)} \u00B1 {round(2 * np.std(ntr), 2)}'
#I believe there are outliers and that my model is overfit

Coefficient values of the Model are  [-0.00285394 -0.00918781  0.00281967  0.365929    2.02752205  0.02784777
  0.30044074  0.47201792 -0.57784028  1.80322559]
Intercept value is  -1.7725794035546514
[0.76724854 0.7737447  0.77479708 0.73886825 0.74067881]


'0.76 ± 0.03'

In [34]:
N = len(G)
p = len(G.columns) + 1  # plus one because LinearRegression adds an intercept term

X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = X.values

beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

ValueError: could not broadcast input array from shape (1682,18) into shape (1682,10)

In [ ]:
y_hat = model.predict(X)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares[0, 0] / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")